# Programming project 2: Harmonic Vibrational analysis

In this project you will be doing a harmonic vibrational analysis on water, benzene and 3-chloro-1-butene. All the necessary input files present in the `input` directory, and you can check your answers with the numerical solutions in the `solutions` folder.

## Step 1: Read the coordinate data

You can use 'Step 1' from Project 1 to read in your coordinate data.

_(Bonus: Can you write a small Python library that provides this functionality so it can be reused?)_

In [ ]:
import pandas as pd
from google.colab import files
import numpy as np
import math

# Read files as dataframe
tri_chloro_1_butene = pd.read_fwf('3_chloro_1_butene.txt', header = None)
water = pd.read_fwf('water.txt', header = None)
benzene = pd.read_fwf('benzene.txt', header = None)

# Reducing dataframes into positions of each atom
tri_chloro_1_butene_pos = tri_chloro_1_butene.loc[1: , 1:]
water_pos = water.loc[1: , 0:]
benzene_pos = benzene.loc[1: , 0:]

# Reducing dataframes into masses of each atom
tri_chloro_1_butene_mass = tri_chloro_1_butene.loc[1: , 1]
water_mass = water.loc[1: , 0]
benzene_mass = benzene.loc[1: , 0]

# Converting to numpy array
tri_chloro_1_butene_pos_np = tri_chloro_1_butene_pos.to_numpy()
water_pos_np = water_pos.to_numpy()
benzene_pos_np = benzene_pos.to_numpy()

tri_chloro_1_butene_mass_np = tri_chloro_1_butene_mass.to_numpy()
water_mass_np = water_mass.to_numpy()
benzene_mass_np = benzene_mass.to_numpy()

## Step 2: Read the cartesian Hessian data

In the directory there are also some `*_hessian.txt` files, which contain a Hessian matrix, which consists of second derivatives of the energy with respect to two atomic positions:

$$F_{ij} = \frac{\partial^2 V}{\partial q_i \partial q_j} \, .$$

The first integer is the number of atoms of the system and the rest of the lines have the following format:

$$\begin{array}{ccc} F_{x_1,x_1} & F_{x_1,y_1} & F_{x_1,z_1} \\ F_{x_1,x_2} & F_{x_1,y_2} & F_{x_1,z_2} \\ & \\ \ldots & \ldots & \dots \\ & \\ F_{x_2,x_1} & F_{x_2,y_1} & F_{x_2,z_1} \\ & \\ \ldots & \ldots & \ldots \end{array} \, .$$

Note that the units of this Hessian matrix are given as $\text{Hartree} \, \text{bohr}^{-2}$.

In [ ]:
import pandas as pd
from google.colab import files
import numpy as np
import math

# Read files as dataframe
tri_chloro_1_butene_hes = pd.read_fwf('3_chloro_1_butene_hessian.txt', header = None)
water_hes = pd.read_fwf('water_hessian.txt', header = None)
benzene_hes = pd.read_fwf('benzene_hessian.txt', header = None)

# Reducing dataframes into positions of each atom
tri_chloro_1_butene_hes = tri_chloro_1_butene_hes.loc[1: , 1:]
water_hes = water_hes.loc[1: , 1:]
benzene_hes = benzene_hes.loc[1: , 1:]

# Converting to numpy array
tri_chloro_1_butene_hes_np = tri_chloro_1_butene_hes.to_numpy()
water_hes_np = water_hes.to_numpy()
benzene_hes_np = benzene_hes.to_numpy()

## Step 3: Mass-weigh the Hessian matrix

The mass-weighted Hessian matrix is calculated by dividing each element of the Hessian matrix by the product of the square roots of the masses of the atoms associated with the given coordinates:

$$F^M_{ij} = \frac{F_{ij}}{\sqrt{m_i m_j}}$$

Like the previous exercise, we are working in amu.


In [ ]:
import numpy as np

# Create matrices of square root of the product of masses
tri_chloro_1_butene_mass_hes = []
for i in range(12):
  for j in range(12):
    tri_chloro_1_butene_mass_hes.append([tri_chloro_1_butene_mass_np[i]*tri_chloro_1_butene_mass_np[j], tri_chloro_1_butene_mass_np[i]*tri_chloro_1_butene_mass_np[j], tri_chloro_1_butene_mass_np[i]*tri_chloro_1_butene_mass_np[j]])
for i in range(12):
  for j in range(12):
    tri_chloro_1_butene_mass_hes.append([tri_chloro_1_butene_mass_np[i]*tri_chloro_1_butene_mass_np[j], tri_chloro_1_butene_mass_np[i]*tri_chloro_1_butene_mass_np[j], tri_chloro_1_butene_mass_np[i]*tri_chloro_1_butene_mass_np[j]])
for i in range(12):
  for j in range(12):
    tri_chloro_1_butene_mass_hes.append([tri_chloro_1_butene_mass_np[i]*tri_chloro_1_butene_mass_np[j], tri_chloro_1_butene_mass_np[i]*tri_chloro_1_butene_mass_np[j], tri_chloro_1_butene_mass_np[i]*tri_chloro_1_butene_mass_np[j]])
tri_chloro_1_butene_mass_hes = np.sqrt(np.asarray(tri_chloro_1_butene_mass_hes))

benzene_mass_hes = []
for i in range(12):
  for j in range(12):
    benzene_mass_hes.append([benzene_mass_np[i]*benzene_mass_np[j], benzene_mass_np[i]*benzene_mass_np[j], benzene_mass_np[i]*benzene_mass_np[j]])
for i in range(12):
  for j in range(12):
    benzene_mass_hes.append([benzene_mass_np[i]*benzene_mass_np[j], benzene_mass_np[i]*benzene_mass_np[j], benzene_mass_np[i]*benzene_mass_np[j]])
for i in range(12):
  for j in range(12):
    benzene_mass_hes.append([benzene_mass_np[i]*benzene_mass_np[j], benzene_mass_np[i]*benzene_mass_np[j], benzene_mass_np[i]*benzene_mass_np[j]])
benzene_mass_hes = np.sqrt(np.asarray(benzene_mass_hes))

water_mass_hes = []
for i in range(3):
  for j in range(3):
    water_mass_hes.append([water_mass_np[i]*water_mass_np[j], water_mass_np[i]*water_mass_np[j], water_mass_np[i]*water_mass_np[j]])
for i in range(3):
  for j in range(3):
    water_mass_hes.append([water_mass_np[i]*water_mass_np[j], water_mass_np[i]*water_mass_np[j], water_mass_np[i]*water_mass_np[j]])
for i in range(3):
  for j in range(3):
    water_mass_hes.append([water_mass_np[i]*water_mass_np[j], water_mass_np[i]*water_mass_np[j], water_mass_np[i]*water_mass_np[j]])
water_mass_hes = np.sqrt(np.asarray(water_mass_hes))

# Divide Hessian matrix by mass matrix
tri_chloro_1_butene_hes_wt = tri_chloro_1_butene_hes_np /tri_chloro_1_butene_mass_hes
water_hes_wt = water_hes_np / water_mass_hes
benzene_hes_wt = benzene_hes_np / benzene_mass_hes

## Step 4: Diagonalize the mass-weighted Hessian matrix

In this step, you should calculate the eigenvalues of the previously calculated mass-weighted Hessian:

$$\mathbf{F}^M \mathbf{L} = \mathbf{L} \Lambda \, .$$

## Step 5: Compute the harmonic vibrational frequencies

Finally, the vibrational frequencies are proportional to the square root of the eigenvalues of the mass-weighted Hessian:

$$\omega_i = \text{constant} \,  \sqrt{\lambda_i} \, .$$

In the solution, frequencies are reported in $\text{cm}^{-1}$.